# Opiod Prescription Project: Classification
* _Data from Kaggle_: https://www.kaggle.com/apryor6/us-opiate-prescriptions  
* _Classification based on Jason Liu's Notebook_: https://www.kaggle.com/jiashenliu/quick-and-dirty-attempt-on-voting-classifier

## 0. Import Libraries and Load Data

### Imports

In [1]:
import pandas as pd  # DataFrame, Series
import numpy as np  # Scientific Computing Packages - Array
import re  # Regularized Expression

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns  # Stats Visualization Library, wraps on top of matplotlib

%matplotlib inline

### Load Data

In [2]:
# Dataframe of opioid drugs and their generic brand equivalents
opioid = pd.read_csv("input/opioids.csv")

# Dataframe of states, total populations, deaths, and state abbrev.
overdose = pd.read_csv("input/overdoses.csv")

# Dataframe of prescribers and features below
prescriber = pd.read_csv("input/prescriber-info.csv")

#### Features in prescriber
* **NPI** – unique National Provider Identifier number  
* **Gender** - (M/F)  
* **State**  - U.S. State by abbreviation  
* **Credentials** - set of initials indicative of medical degree  
* **Specialty** - description of type of medicinal practice  
* A long list of drugs with numeric values indicating the total number of prescriptions written for the year by that individual  
* **Opioid.Prescriber** - a boolean label indicating whether or not that individual prescribed opiate drugs more than 10 times in the year

## I. Pre-processing
For classification, we will transform the features into all numeric values

In [3]:
# Full list of all opioid drugs
name = opioid['Drug Name']

# Rename without spaces or "-"s
new_name = name.apply(lambda x: re.sub("/ |-", ".", str(x)))

# List of all drugs that were prescribed, NPI, Gender, State, Specialty, etc
col = prescriber.columns

abandoned_var = set(col).intersection(set(new_name))
keep_var = []

# keep_var holds all opioids that are actually prescribed and categorical data
for each in col:
    if each in abandoned_var:
        pass
    else:
        keep_var.append(each)
        
pres = prescriber[keep_var]

### Train and Test Split

In [4]:
# 80/20 train-test split
train, test = train_test_split(pres, test_size=0.2, random_state=42)

In [5]:
# test should be 1/4 (20/80) the size of train
print(train.shape)
print(test.shape)

(20000, 251)
(5000, 251)


In [6]:
category = ['Gender', 'State', 'Credentials', 'Specialty']

for col in category:
    train[col] = pd.factorize(train[col], sort=True)[0]
    test[col] = pd.factorize(test[col], sort=True)[0]

/Users/anepomu1/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/anepomu1/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Features

In [7]:
features = train.iloc[:, 1:251]
features.head()

,Gender,State,Credentials,Specialty,ABILIFY,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,ALLOPURINOL,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
23311,1,11,594,62,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23623,0,24,196,23,0,0,33,0,0,19,...,0,0,0,0,15,0,0,0,53,1
1020,1,47,431,39,0,0,0,0,64,62,...,16,0,14,0,128,0,11,0,71,1
12645,1,37,431,103,0,0,0,0,0,0,...,0,22,0,0,0,0,0,0,0,0
1533,1,55,344,66,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## III. Try Different Classifiers

### Import everything

In [8]:
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier

### Prepare for Models

In [9]:
# Repeat features from above
features = train.iloc[:, 1:251]

# What are we training for?
target = train['Opioid.Prescriber']

# Names and accuracies of models to be tested
Name = []
Accuracy = []

### Classification Models to Compare

In [12]:
m1 = LogisticRegression(random_state=22, C=0.000000001, solver='liblinear', max_iter=200)
m2 = GaussianNB()
m3 = RandomForestClassifier(n_estimators=200, random_state=22)
m4 = GradientBoostingClassifier(n_estimators=200)
m5 = KNeighborsClassifier()
m6 = DecisionTreeClassifier()
m7 = LinearDiscriminantAnalysis()

### Assess Models' Accuracy Performances

In [16]:
# Create an ensemble and score them based on accuracy to vote for the 
    # best classifier
Ensemble_model = VotingClassifier(estimators=[('lr', m1), 
                                              ('gn', m2), 
                                              ('rf', m3), 
                                              ('gb', m4), 
                                              ('kn', m5), 
                                              ('dt', m6), 
                                              ('lda', m7)], 
                                  voting='hard')

for model, label in zip([m1, m2, m3, m4, m5, m6, m7, Ensemble_model],
                       ['Logistic Regression', 'Naive Bayes', 'Random Forest', 
                        'Gradient Boosting', 'KNN', 'Decision Tree', 'LDA', 
                        'Ensemble']):
    scores = cross_val_score(model, features, target, cv=5, scoring='accuracy')
    Accuracy.append(scores.mean())
    Name.append(model.__class__.__name__)
    print("Accuracy: %f of model %s" % (scores.mean(),label))

Accuracy: 0.589000 of model Logistic Regression
Accuracy: 0.999850 of model Naive Bayes
Accuracy: 0.999800 of model Random Forest
Accuracy: 1.000000 of model Gradient Boosting
Accuracy: 0.769900 of model KNN
Accuracy: 1.000000 of model Decision Tree


/Users/anepomu1/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/anepomu1/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/anepomu1/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/anepomu1/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/anepomu1/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Accuracy: 0.723000 of model LDA


/Users/anepomu1/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/anepomu1/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/anepomu1/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/anepomu1/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/anepomu1/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Accuracy: 0.999800 of model Ensemble


### Drop Models with Accuracy below 80%, re-ensemble the model

In [24]:
Name_2 = []
Accuracy_2 = []

Ensemble_model_2 = VotingClassifier(estimators=[('gn', m2), 
                                                ('rf', m3), 
                                                ('gb', m4),  
                                                ('dt', m6)],  
                                      voting='hard')

for model, label in zip([m2, m3, m4, m6, Ensemble_model_2], 
                        ['Naive Bayes', 'Random Forest', 
                         'Gradient Boosting', 'Decision Tree', 
                         'Ensemble']):
    scores = cross_val_score(model, features, target, cv=5, scoring='accuracy')
    Accuracy_2.append(scores.mean())
    Name_2.append(model.__class__.__name__)
    print("Accuracy: %f of model %s" % (scores.mean(),label))

Accuracy: 0.999850 of model Naive Bayes
Accuracy: 0.999800 of model Random Forest
Accuracy: 1.000000 of model Gradient Boosting
Accuracy: 1.000000 of model Decision Tree
Accuracy: 1.000000 of model Ensemble


### Evaluate with Test Set

In [26]:
from sklearn.metrics import accuracy_score

classifiers = [m2, m3, m4, m6, Ensemble_model_2]
out_sample_accuracy = []
Name_2 = []

for each in classifiers:
    fit = each.fit(features,target)
    pred = fit.predict(test.iloc[:, 1:244])
    accuracy = accuracy_score(test['Opioid.Prescriber'], pred)
    Name_2.append(each.__class__.__name__)
    out_sample_accuracy.append(accuracy)

ValueError: operands could not be broadcast together with shapes (5000,243) (250,) 